## Даты последних изменений маршрутов общественного транспорта с сайта WikiRoutes

Сбор данных о дате пользовательских изменений маршрутов общественного транспорта на сайте [wikiroutes.info](https://wikiroutes.info/) (в ознакомительных целях).

In [2]:
# Устанавливаем библиотеку playwright
!pip install playwright nest_asyncio


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [57]:
# Устанавливаем библиотеку для работы с Google Sheets API
#!pip install gspread
import gspread

In [4]:
# Устанавливаем браузер для работы playwright
!playwright install

In [1]:
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright

import gspread
import json
import pandas as pd
import re

from datetime import datetime

nest_asyncio.apply()

Работа с данным заключается в сборе информации с веб-страницы. На ней представлены данные о маршруте общественного транспорта, а также дата внесения изменений пользователями.

Отслеживание этой информации позволяет определить, вносились ли изменения в маршрут.

In [ ]:
# Создание учётной записи в Google Sheets API для нашей программы
# https://docs.gspread.org/en/v6.1.3/oauth2.html#enable-api-access
# https://www.youtube.com/watch?v=RmEsC2T8dwE
# Читаем файл json с параметрами
key_file = json.load(open("///.json"))

In [ ]:
# Подключаем сервисный аккаунт Google Sheets API
gc = gspread.service_account(filename="///.json")

# Открываем нужную Гугл-таблицу (первый лист)
wks = gc.open("Ссылки на маршруты WikiRoutes").sheet1

# Забираем данные (ссылки) из 1-го столбца, начиная со 2-й строки (включая пустые строки между заполненными ячейками)
link_sheet = {i: {link: ""} for i, link in enumerate(wks.col_values(1)[1:], 2)}
link_dict = link_sheet.copy()

#link_dict = dict(enumerate(wks.col_values(1)[1:], 2))

In [48]:
link_dict

{2: {'https://wikiroutes.info/klin?routes=27396': ''},
 3: {'https://wikiroutes.info/reutov?routes=10209': ''},
 4: {'https://wikiroutes.info/chehov?routes=56264': ''},
 5: {'https://wikiroutes.info/msk?routes=30002': ''},
 6: {'https://wikiroutes.info/lyubertsy?routes=6538': ''},
 7: {'https://wikiroutes.info/solnechnogorsk?routes=6025': ''},
 8: {'https://wikiroutes.info/mendeleyevo?routes=8830': ''},
 9: {'https://wikiroutes.info/ramenskoe?routes=9655': ''},
 10: {'https://wikiroutes.info/msk?routes=4835': ''},
 11: {'https://wikiroutes.info/goluboye?routes=42319': ''},
 12: {'https://wikiroutes.info/khimki?routes=1036': ''},
 13: {'https://wikiroutes.info/dmitrov?routes=12762': ''}}

In [49]:
# Проверяем количество полученных ссылок
len(link_dict)

12

## Первичный сбор данных

In [ ]:
async def run_playwright():  # создаём асинхронную функцию
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)

        for i, v in link_dict.items():  # проходим по сслыкам из полученного словаря
            for link, date in v.items():
                if link != "":  # если элемент в списке ссылок не пустой, то происходит переход по ссылке
                    try:
                        # создаём новую страницу (вкладку) для каждой ссылки
                        page = await browser.new_page()
                        await page.goto(link)
                        await page.wait_for_timeout(3000)

                        # выбираем локатор по роли "ссылка", в которой находится дата (поиск с помощью регулярных выражений)
                        last_edited_locator = page.get_by_role("link", name=re.compile(r"\d{2}\.\d{2}\.\d{4}"))
                        # забираем текст из выбранного элемента
                        last_edited_text = await last_edited_locator.inner_text()
                        print(f"1 - {last_edited_text}")

                        # забираем только дату из полученного текста
                        match = re.search(r"(\d{2}\.\d{2}\.\d{4})", last_edited_text)
                        
                        # если дата успешно получена, добавляем полученный результат в список, где собираются даты
                        if match:
                            extracted_date = match.group(1)
                            print(f"2 - {extracted_date}")
                            v[link] = extracted_date
                        # если дата не получена, делаем запись об отсутствии данных
                        else:
                            print("Нет данных")
                            v[link] = "Нет данных"
                    # если не получилось собрать данные со страницы, делаем запись об отсутствии данных
                    except Exception as e:
                        print(f"Ошибка: {e}")
                        v[link] = "Нет данных"
                    finally:
                        # закрываем текущую страницу после обработки
                        await page.wait_for_timeout(3000)
                        await page.close()
                # если элемент, в котором должна быть ссылка, пустой, тогда элемент с датой также делаем пустым
                else:
                    v[link] = ""
        return link_dict
        # Закрываем браузер
        await browser.close()

import asyncio
asyncio.run(run_playwright())

Ошибка: Timeout 30000ms exceeded.
1 - 1 мес назад или 07.10.2024
2 - 07.10.2024
1 - 3 мес назад или 23.07.2024
2 - 23.07.2024
1 - 4 мес назад или 20.07.2024
2 - 20.07.2024
1 - 3 мес назад или 23.07.2024
2 - 23.07.2024
1 - 3 мес назад или 23.07.2024
2 - 23.07.2024
1 - 3 мес назад или 23.07.2024
2 - 23.07.2024
1 - 3 мес назад или 23.07.2024
2 - 23.07.2024
Ошибка: Timeout 30000ms exceeded.
1 - 3 июля 2022 или 03.07.2022
2 - 03.07.2022
1 - 3 мес назад или 23.07.2024
2 - 23.07.2024
1 - 20 дн назад или 30.10.2024
2 - 30.10.2024


{2: {'': ''},
 3: {'': ''},
 4: {'https://wikiroutes.info/klin?routes=27396': 'Нет данных'},
 5: {'https://wikiroutes.info/reutov?routes=10209': '07.10.2024'},
 6: {'https://wikiroutes.info/chehov?routes=56264': '23.07.2024'},
 7: {'https://wikiroutes.info/msk?routes=30002': '20.07.2024'},
 8: {'https://wikiroutes.info/lyubertsy?routes=6538': '23.07.2024'},
 9: {'https://wikiroutes.info/solnechnogorsk?routes=6025': '23.07.2024'},
 10: {'https://wikiroutes.info/mendeleyevo?routes=8830': '23.07.2024'},
 11: {'https://wikiroutes.info/ramenskoe?routes=9655': '23.07.2024'},
 12: {'https://wikiroutes.info/msk?routes=4835': 'Нет данных'},
 13: {'': ''},
 14: {'https://wikiroutes.info/goluboye?routes=42319': '03.07.2022'},
 15: {'https://wikiroutes.info/khimki?routes=1036': '23.07.2024'},
 16: {'': ''},
 17: {'': ''},
 18: {'': ''},
 19: {'': ''},
 20: {'https://wikiroutes.info/dmitrov?routes=12762': '30.10.2024'}}

In [13]:
link_dict

{2: {'': ''},
 3: {'': ''},
 4: {'https://wikiroutes.info/klin?routes=27396': 'Нет данных'},
 5: {'https://wikiroutes.info/reutov?routes=10209': '07.10.2024'},
 6: {'https://wikiroutes.info/chehov?routes=56264': '23.07.2024'},
 7: {'https://wikiroutes.info/msk?routes=30002': '20.07.2024'},
 8: {'https://wikiroutes.info/lyubertsy?routes=6538': '23.07.2024'},
 9: {'https://wikiroutes.info/solnechnogorsk?routes=6025': '23.07.2024'},
 10: {'https://wikiroutes.info/mendeleyevo?routes=8830': '23.07.2024'},
 11: {'https://wikiroutes.info/ramenskoe?routes=9655': '23.07.2024'},
 12: {'https://wikiroutes.info/msk?routes=4835': 'Нет данных'},
 13: {'': ''},
 14: {'https://wikiroutes.info/goluboye?routes=42319': '03.07.2022'},
 15: {'https://wikiroutes.info/khimki?routes=1036': '23.07.2024'},
 16: {'': ''},
 17: {'': ''},
 18: {'': ''},
 19: {'': ''},
 20: {'https://wikiroutes.info/dmitrov?routes=12762': '30.10.2024'}}

## Дополнительный сбор данных

Если не у всех маршрутов удалось подгрузить даты, пробуем собрать данные у оставшихся маршрутов.

In [16]:
async def run_playwright():  # создаём асинхронную функцию
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)

        for i, v in link_dict.items():  # проходим по сслыкам из полученного словаря
            for link, date in v.items():
                if date == "Нет данных":  # если элемент в списке не пустой, то происходит переход по ссылке
                    try:
                        # Создаём новую страницу (вкладку) для каждой ссылки
                        page = await browser.new_page()
                        await page.goto(link)
                        await page.wait_for_timeout(3000)

                        # выбираем локатор по роли "ссылка", в которой находится дата (поиск с помощью регулярных выражений)
                        last_edited_locator = page.get_by_role("link", name=re.compile(r"\d{2}\.\d{2}\.\d{4}"))
                        # забираем текст из выбранного элемента
                        last_edited_text = await last_edited_locator.inner_text()
                        print(f"1 - {last_edited_text}")

                        # забираем только дату из полученного текста
                        match = re.search(r"(\d{2}\.\d{2}\.\d{4})", last_edited_text)
                        
                        # если дата успешно получена, добавляем полученный результат в список, где собираются даты
                        if match:
                            extracted_date = match.group(1)
                            print(f"2 - {extracted_date}")
                            v[link] = extracted_date   
                        # если дата не получена, делаем запись об отсутствии данных
                        else:
                            print("Нет данных")
                            v[link] = "Нет данных"
                    # если не получилось собрать данные со страницы, делаем запись об отсутствии данных
                    except Exception as e:
                        print(f"Ошибка: {e}")
                        v[link] = "Нет данных"
                    finally:
                        # Закрываем текущую страницу после обработки
                        await page.wait_for_timeout(3000)
                        await page.close()
                else:
                    continue
        return link_dict
        # Закрываем браузер
        await browser.close()

import asyncio
asyncio.run(run_playwright())

1 - 1 мес назад или 14.10.2024
2 - 14.10.2024


{2: {'': ''},
 3: {'': ''},
 4: {'https://wikiroutes.info/klin?routes=27396': '15.08.2022'},
 5: {'https://wikiroutes.info/reutov?routes=10209': '07.10.2024'},
 6: {'https://wikiroutes.info/chehov?routes=56264': '23.07.2024'},
 7: {'https://wikiroutes.info/msk?routes=30002': '20.07.2024'},
 8: {'https://wikiroutes.info/lyubertsy?routes=6538': '23.07.2024'},
 9: {'https://wikiroutes.info/solnechnogorsk?routes=6025': '23.07.2024'},
 10: {'https://wikiroutes.info/mendeleyevo?routes=8830': '23.07.2024'},
 11: {'https://wikiroutes.info/ramenskoe?routes=9655': '23.07.2024'},
 12: {'https://wikiroutes.info/msk?routes=4835': '14.10.2024'},
 13: {'': ''},
 14: {'https://wikiroutes.info/goluboye?routes=42319': '03.07.2022'},
 15: {'https://wikiroutes.info/khimki?routes=1036': '23.07.2024'},
 16: {'': ''},
 17: {'': ''},
 18: {'': ''},
 19: {'': ''},
 20: {'https://wikiroutes.info/dmitrov?routes=12762': '30.10.2024'}}

In [44]:
link_dict

{2: {'': ''},
 3: {'': ''},
 4: {'https://wikiroutes.info/klin?routes=27396': ''},
 5: {'https://wikiroutes.info/reutov?routes=10209': ''},
 6: {'https://wikiroutes.info/chehov?routes=56264': ''},
 7: {'https://wikiroutes.info/msk?routes=30002': ''},
 8: {'https://wikiroutes.info/lyubertsy?routes=6538': ''},
 9: {'https://wikiroutes.info/solnechnogorsk?routes=6025': ''},
 10: {'https://wikiroutes.info/mendeleyevo?routes=8830': ''},
 11: {'https://wikiroutes.info/ramenskoe?routes=9655': ''},
 12: {'https://wikiroutes.info/msk?routes=4835': ''},
 13: {'': ''},
 14: {'https://wikiroutes.info/goluboye?routes=42319': ''},
 15: {'https://wikiroutes.info/khimki?routes=1036': ''},
 16: {'': ''},
 17: {'': ''},
 18: {'': ''},
 19: {'': ''},
 20: {'https://wikiroutes.info/dmitrov?routes=12762': ''}}

In [18]:
# Проверяем количество полученных ссылок
len(link_dict)

19

In [41]:
# Для добавления данных в Гугл-таблицу
# создаём отдельный список, куда соберём все полученные даты и пропуски (если они имеются)
date_list = [l for link in link_dict.values() for l in link.values()]

(['',
  '',
  '15.08.2022',
  '07.10.2024',
  '23.07.2024',
  '20.07.2024',
  '23.07.2024',
  '23.07.2024',
  '23.07.2024',
  '23.07.2024',
  '14.10.2024',
  '',
  '03.07.2022',
  '23.07.2024',
  '',
  '',
  '',
  '',
  '30.10.2024'],
 19)

In [45]:
# Задаём переменную с размерами строк и столбцов, по которой будут вноситься данные на лист таблицы
cell_range = f"F2:F{len(link_dict) + 1}"
cell_list = wks.range(cell_range)

# С помощью индексированного цикла передаём элементы списка с данными в подготовленный шаблон
# с заданными параметрами строк и столбцов
for i, cell in enumerate(cell_list):
    cell.value = date_list[i]

# Передаём подготовленные данные в таблицу
wks.update_cells(cell_list)

{'spreadsheetId': '1c0MwtlJndmZyErff83ppd51er8Piwd_GuIhJ2Agyn7w',
 'updatedRange': 'Links!F2:F20',
 'updatedRows': 19,
 'updatedColumns': 1,
 'updatedCells': 19}